In [10]:
from interpreter import interpreter
import os

In [11]:
interpreter.llm.model = "azure/gpt-4.1"
interpreter.llm.api_key = os.getenv("AZURE_OPENAI_API_KEY")
interpreter.llm.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
interpreter.llm.api_version = os.getenv("AZURE_OPENAI_API_VERSION")
interpreter.auto_run = True

output = interpreter.chat("using pyautogui, press ctrl + t to open a new tab in the browser")

print(output)

c:\Users\sathish\miniconda3\envs\py3.10\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[{'role': 'assistant', 'type': 'code', 'format': 'python', 'content': 'import pyautogui\nimport time\n\n# Wait for a short moment to let user focus the browser\nprint("Switch to your browser window in the next 3 seconds...")\ntime.sleep(3)\n\n# Press Ctrl+T (Windows and Linux)\npyautogui.hotkey(\'ctrl\', \'t\')\nprint(\'Pressed Ctrl+T to open a new tab.\')'}, {'role': 'computer', 'type': 'console', 'format': 'output', 'content': 'Switch to your browser window in the next 3 seconds...\nPressed Ctrl+T to open a new tab.\n'}, {'role': 'assistant', 'type': 'message', 'content': 'The Ctrl+T hotkey has been sent using pyautogui. If your browser window was in focus, you should see a new tab opened now.\n\nLet me know if you need to automate any other tasks!'}]


In [17]:
import os
from autogen import AssistantAgent, UserProxyAgent

# Azure OpenAI configuration
azure_config = {
    "model": "gpt-4.1",  # use deployment name, not model name
    "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
    "api_type": "azure",
    "base_url": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
}

# Create an assistant agent using Azure
assistant = AssistantAgent(
    name="azure_agent",
    llm_config=azure_config,
)

# Create a user proxy agent
user = UserProxyAgent(name="user", code_execution_config={"use_docker": False})

# Start a conversation
user.initiate_chat(assistant, message="Write a Python function that sorts a list of numbers.")

user (to azure_agent):

Write a Python function that sorts a list of numbers.

--------------------------------------------------------------------------------
azure_agent (to user):

Plan:
- I'll write a Python function named sort_numbers that takes a list of numbers as its argument and returns a sorted version of the list in ascending order.
- I'll use Python's built-in sorted() function for simplicity and reliability.
- I'll output this code so you can execute it and verify.

Here's the code:

```python
def sort_numbers(numbers):
    """
    Sorts a list of numbers in ascending order.

    :param numbers: List of numeric values (int or float)
    :return: Sorted list of numbers
    """
    return sorted(numbers)

# Example usage
numbers = [5, 2, 9, 1, 5, 6]
sorted_numbers = sort_numbers(numbers)
print("Sorted numbers:", sorted_numbers)
```

Please execute the code to see the sorted list. Let me know if you'd like to sort in descending order or have more requirements! TERMINATE

----

ChatResult(chat_id=126235860762959170183215105791882409728, chat_history=[{'content': 'Write a Python function that sorts a list of numbers.', 'role': 'assistant', 'name': 'user'}, {'content': 'Plan:\n- I\'ll write a Python function named sort_numbers that takes a list of numbers as its argument and returns a sorted version of the list in ascending order.\n- I\'ll use Python\'s built-in sorted() function for simplicity and reliability.\n- I\'ll output this code so you can execute it and verify.\n\nHere\'s the code:\n\n```python\ndef sort_numbers(numbers):\n    """\n    Sorts a list of numbers in ascending order.\n\n    :param numbers: List of numeric values (int or float)\n    :return: Sorted list of numbers\n    """\n    return sorted(numbers)\n\n# Example usage\nnumbers = [5, 2, 9, 1, 5, 6]\nsorted_numbers = sort_numbers(numbers)\nprint("Sorted numbers:", sorted_numbers)\n```\n\nPlease execute the code to see the sorted list. Let me know if you\'d like to sort in descending order or 